In [336]:
import re
from json import dump
import requests

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 51) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)


for page in N_PAGES:
    url = BASE_URL + f"/rent/?bedrooms=1-any&state=vic&page={page}"
    bs_object = BeautifulSoup(requests.get(
    url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])





In [338]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    # bs_object = BeautifulSoup(urlopen(property_url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    if(bs_object.find("h1", {"class": "css-164r41r"})is None):
        continue;
    else:
        property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}).text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    house_type = bs_object.find("div", {"data-testid": "listing-summary-property-type"})
    if house_type :
        property_metadata[property_url]['house_type'] = house_type.text
    else: 
        property_metadata[property_url]['house_type'] = None


    information = bs_object.find("div", {"data-testid": "strip-content-list"})
    if information :
        property_metadata[property_url]['external_information'] = information.text
    else: 
        property_metadata[property_url]['external_information'] = None

    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/
with open('../data/raw/example.json', 'w') as f:
    dump(property_metadata, f)

In [341]:
# begin code
url_links = []

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?bedrooms=1-any&page={page}"
    bs_object = BeautifulSoup(requests.get(
    url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])




In [345]:
property_metadata = defaultdict(dict)
#  for each url, scrape some basic metadata
for property_url in url_links:
    # bs_object = BeautifulSoup(urlopen(property_url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    if(bs_object.find("h1", {"class": "css-164r41r"})is None):
        continue;
    else:
        property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}).text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    house_type = bs_object.find("div", {"data-testid": "listing-summary-property-type"})
    if house_type :
        property_metadata[property_url]['house_type'] = house_type.text
    else: 
        property_metadata[property_url]['house_type'] = None
    
    information = bs_object.find("div", {"data-testid": "strip-content-list"})
    if information :
        property_metadata[property_url]['external_information'] = information.text
    else: 
        property_metadata[property_url]['external_information'] = None

    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/
with open('../data/raw/example2.json', 'w') as f:
    dump(property_metadata, f)

In [346]:
# begin code
url_links = []

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/?ptype=house&state=vic&page={page}"
    bs_object = BeautifulSoup(requests.get(
    url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])

In [347]:
property_metadata = defaultdict(dict)
#  for each url, scrape some basic metadata
for property_url in url_links:
    # bs_object = BeautifulSoup(urlopen(property_url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    if(bs_object.find("h1", {"class": "css-164r41r"})is None):
        continue;
    else:
        property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}).text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    house_type = bs_object.find("div", {"data-testid": "listing-summary-property-type"})
    if house_type :
        property_metadata[property_url]['house_type'] = house_type.text
    else: 
        property_metadata[property_url]['house_type'] = None
    
    information = bs_object.find("div", {"data-testid": "strip-content-list"})
    if information :
        property_metadata[property_url]['external_information'] = information.text
    else: 
        property_metadata[property_url]['external_information'] = None

    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/
with open('../data/raw/example3.json', 'w') as f:
    dump(property_metadata, f)

In [352]:
import json
import pandas as pd
  
# Opening JSON file
f = open('../data/raw/example.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
house_data = pd.DataFrame.from_dict(data, orient="index")
house_data

,name,cost_text,coordinates,rooms,house_type,external_information,desc
https://www.domain.com.au/188b-kerferd-road-albert-park-vic-3206-16067878,188B Kerferd Road Albert Park VIC 3206,$670.00 per week,"[-37.8479747, 144.9520365]","[[1 Bed], [1 Bath], []]",Apartment / Unit / Flat,"Available from Friday, 9th September 2022",Nadia Barnes
https://www.domain.com.au/3-34-leslie-street-richmond-vic-3121-16067868,3/34 Leslie Street Richmond VIC 3121,$320 per week,"[-37.8132125, 145.0067341]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,"Available from Wednesday, 5th October 2022Bond...",Ella Vander Sluys
https://www.domain.com.au/20-12-dundas-street-thornbury-vic-3071-15981310,20/12 Dundas Street Thornbury VIC 3071,$240.00,"[-37.7528368, 145.0029462]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $1043,Kathryn Ambesi
https://www.domain.com.au/4-317-dandenong-road-prahran-vic-3181-16067845,4/317 Dandenong Road Prahran VIC 3181,$430 Per Week,"[-37.859496, 145.0083698]","[[2 Beds], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,"Available from Friday, 9th September 2022Bond ...",South Melbourne Leasing
https://www.domain.com.au/43-philipson-street-albert-park-vic-3206-16067830,43 Philipson Street Albert Park VIC 3206,$680.00,"[-37.8479147, 144.9506225]","[[3 Beds], [1 Bath], []]",House,"Available from Thursday, 29th September 2022Bo...",Louisa Corke
...,...,...,...,...,...,...,...
https://www.domain.com.au/6b-168-victoria-road-northcote-vic-3070-16044728,6b/168 Victoria Road Northcote VIC 3070,$370.00,"[-37.7703059, 145.0119705]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,"Available from Friday, 30th September 2022Bond...",Alicia Cannizzaro
https://www.domain.com.au/6-65-clarence-street-elsternwick-vic-3185-16039179,6/65 Clarence Street Elsternwick VIC 3185,$450 weekly,"[-37.8926328, 145.0127778]","[[2 Beds], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $1800,Susie Bourke
https://www.domain.com.au/14-37-vale-st-st-kilda-vic-3182-16044667,14/37 Vale St St Kilda VIC 3182,$315 weekly,"[-37.8665707, 144.9847763]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $1369,Lauren Allingham
https://www.domain.com.au/925-555-flinders-street-melbourne-vic-3000-16044643,925/555 Flinders Street Melbourne VIC 3000,$500.00 per week,"[-37.8209711, 144.9564827]","[[2 Beds], [2 Baths], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $2173,Barbara Baltas


In [348]:
import json
import pandas as pd
  
# Opening JSON file
f2 = open('../data/raw/example2.json')
  
data_2 = json.load(f2)
house_data_2 = pd.DataFrame.from_dict(data_2, orient="index")
house_data_2

,name,cost_text,coordinates,rooms,house_type,external_information,desc
https://www.domain.com.au/3-26-wynnstay-road-prahran-vic-3181-15995682?topspot=1,3/26 Wynnstay Road Prahran VIC 3181,$350 Per week,"[-37.8529251, 145.0100271]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $1521,Tom Koller
https://www.domain.com.au/38-urwin-street-yarraville-vic-3013-16068181,38 Urwin Street Yarraville VIC 3013,$550.00 per week,"[-37.8171671, 144.8651073]","[[3 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available Now,Nadia Barnes
https://www.domain.com.au/7-gillyweed-avenue-clyde-north-vic-3978-14601307,7 Gillyweed Avenue Clyde North VIC 3978,$470 per week ($2042 pcm),"[-38.110329, 145.3274613]","[[4 Beds], [2 Baths], [2 Parking]]",House,"Available from Monday, 5th September 2022Bond ...",Kavinda Kudage Don
https://www.domain.com.au/102-729-sydney-road-coburg-vic-3058-16068134,102/729 Sydney Road Coburg VIC 3058,$420.00,"[-37.7342626, 144.9653179]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,"Available from Thursday, 29th September 2022Bo...",Helena Eglezos
https://www.domain.com.au/10-wenden-grove-st-kilda-east-vic-3183-16068114,10 Wenden Grove St Kilda East VIC 3183,$900 weekly,"[-37.8667339, 145.0075519]","[[3 Beds], [2 Baths], [2 Parking]]",Townhouse,"Available from Saturday, 3rd September 2022Bon...",Leasing Concierge
...,...,...,...,...,...,...,...
https://www.domain.com.au/4-36-blessington-street-st-kilda-vic-3182-16022078,4/36 Blessington Street St Kilda VIC 3182,$300 per week,"[-37.8697373, 144.9819009]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $1303,Xavier George
https://www.domain.com.au/109-easey-street-collingwood-vic-3066-16021861,109 Easey Street Collingwood VIC 3066,$750,"[-37.7983554, 144.9907584]","[[3 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $3259,Nigel Chee
https://www.domain.com.au/4-palfreyman-street-south-yarra-vic-3141-16021762,4 Palfreyman Street South Yarra VIC 3141,$980 Per Week,"[-37.844978, 144.997811]","[[3 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $4900,Tom Koller
https://www.domain.com.au/5e-635-st-kilda-rd-melbourne-vic-3004-16020842,5E/635 St Kilda Rd Melbourne VIC 3004,$750,"[-37.854087, 144.9824809]","[[2 Beds], [2 Baths], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $3259,Sabrina Song


In [349]:
import json
import pandas as pd
  
# Opening JSON file
f2 = open('../data/raw/example3.json')
  
data_3 = json.load(f2)
house_data_3 = pd.DataFrame.from_dict(data_3, orient="index")
house_data_3

,name,cost_text,coordinates,rooms,house_type,external_information,desc
https://www.domain.com.au/13-highland-crescent-wodonga-vic-3690-16025538?topspot=1,13 Highland Crescent Wodonga VIC 3690,$430,"[-36.1201422, 146.8651548]","[[3 Beds], [1 Bath], [3 Parking], []]",House,"Available from Tuesday, 1st November 2022Bond ...",Bree Sinclair
https://www.domain.com.au/40-kelvin-grove-prahran-vic-3181-16068985,40 Kelvin Grove Prahran VIC 3181,$815 per week,"[-37.8568254, 145.0088863]","[[3 Beds], [1 Bath], [2 Parking]]",House,"Available from Thursday, 15th September 2022Bo...",Joshua Ling
https://www.domain.com.au/40-smith-street-thornbury-vic-3071-16068924,40 Smith Street Thornbury VIC 3071,$650 per week,"[-37.75718030000001, 144.9970065]","[[2 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $2824,Georgia O'Gorman
https://www.domain.com.au/4-mcpherson-street-brunswick-vic-3056-16068777,4 McPherson Street Brunswick VIC 3056,$750.00,"[-37.7580441, 144.9725205]","[[3 Beds], [2 Baths], []]",House,"Available from Monday, 12th September 2022Bond...",Kathryn Ambesi
https://www.domain.com.au/13-martin-street-south-melbourne-vic-3205-16068578,13 Martin Street South Melbourne VIC 3205,"$1,100.00","[-37.8370147, 144.9585055]","[[3 Beds], [2 Baths], [2 Parking]]",House,"Available from Tuesday, 4th October 2022Bond $...",Annie Gillan
...,...,...,...,...,...,...,...
https://www.domain.com.au/1-4-victor-street-sunshine-north-vic-3020-16022777,1/4 Victor Street Sunshine North VIC 3020,$370,"[-37.7777696, 144.8437958]","[[3 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $1607,This attractive home in a great location in Su...
https://www.domain.com.au/24-david-street-brunswick-vic-3056-16022755,24 David Street Brunswick VIC 3056,$575pw / $2499pcm,"[-37.7704915, 144.9627108]","[[2 Beds], [1 Bath], []]",House,Date Available: Available NowBond $2499,This spacious home offers a range of impressiv...
https://www.domain.com.au/5-wareham-street-springvale-vic-3171-16022674,5 Wareham Street Springvale VIC 3171,$515 Per Week,"[-37.9385223, 145.1558214]","[[3 Beds], [1 Bath], [2 Parking]]",House,Date Available: Available NowBond $2238,Positioned within walking distance to Springva...
https://www.domain.com.au/3-34-hudson-street-coburg-vic-3058-16022635,3/34 Hudson Street Coburg VIC 3058,$390 p/w,"[-37.7422322, 144.9628022]","[[2 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $1690,Centrally located in the heart of Coburg sits ...


In [350]:
def filter_dataset(df):
    l = []
    for i in range(len(df)):
        if(len(df['rooms'][i]) != 0):
            l.append(True)
        else:
            l.append(False)
    df = df[l]
    return df

In [354]:
house_data = filter_dataset(house_data)
house_data_2 = filter_dataset(house_data_2)
house_data_3 = filter_dataset(house_data_3)

In [355]:
house_data_combined = pd.concat([house_data, house_data_2, house_data_3])

In [356]:
house_data_combined = house_data_combined.reset_index()
house_data_combined.rename(columns={'index':'link'}, inplace = True)

In [357]:
import re
bedroom_list = []
bathroom_list = []
parking_list = []
for i in range(len(house_data_combined)):
    bedroom_list.append(re.search('\d+', house_data_combined['rooms'][i][0][0]).group())
    bathroom_list.append(re.search('\d+', house_data_combined['rooms'][i][1][0]).group())
    if(len(house_data_combined['rooms'][i][2])!= 0):
        parking_list.append(re.search('\d+', house_data_combined['rooms'][i][2][0]).group())
    else:
        parking_list.append('0')
    # df3['bathroom'][i] = re.search('\d+', df3['rooms'][i][1][0]).group()
    # df3['parking'][i] = re.search('\d+', df3['rooms'][i][2][0]).group()

In [358]:
house_data_combined['bedroom'] = bedroom_list
house_data_combined['bathroom'] = bathroom_list
house_data_combined['parking'] = parking_list

In [359]:
house_data_combined

,link,name,cost_text,coordinates,rooms,house_type,external_information,desc,bedroom,bathroom,parking
0,https://www.domain.com.au/188b-kerferd-road-al...,188B Kerferd Road Albert Park VIC 3206,$670.00 per week,"[-37.8479747, 144.9520365]","[[1 Bed], [1 Bath], []]",Apartment / Unit / Flat,"Available from Friday, 9th September 2022",Nadia Barnes,1,1,0
1,https://www.domain.com.au/3-34-leslie-street-r...,3/34 Leslie Street Richmond VIC 3121,$320 per week,"[-37.8132125, 145.0067341]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,"Available from Wednesday, 5th October 2022Bond...",Ella Vander Sluys,1,1,1
2,https://www.domain.com.au/20-12-dundas-street-...,20/12 Dundas Street Thornbury VIC 3071,$240.00,"[-37.7528368, 145.0029462]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $1043,Kathryn Ambesi,1,1,1
3,https://www.domain.com.au/4-317-dandenong-road...,4/317 Dandenong Road Prahran VIC 3181,$430 Per Week,"[-37.859496, 145.0083698]","[[2 Beds], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,"Available from Friday, 9th September 2022Bond ...",South Melbourne Leasing,2,1,1
4,https://www.domain.com.au/43-philipson-street-...,43 Philipson Street Albert Park VIC 3206,$680.00,"[-37.8479147, 144.9506225]","[[3 Beds], [1 Bath], []]",House,"Available from Thursday, 29th September 2022Bo...",Louisa Corke,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2997,https://www.domain.com.au/1-4-victor-street-su...,1/4 Victor Street Sunshine North VIC 3020,$370,"[-37.7777696, 144.8437958]","[[3 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $1607,This attractive home in a great location in Su...,3,1,1
2998,https://www.domain.com.au/24-david-street-brun...,24 David Street Brunswick VIC 3056,$575pw / $2499pcm,"[-37.7704915, 144.9627108]","[[2 Beds], [1 Bath], []]",House,Date Available: Available NowBond $2499,This spacious home offers a range of impressiv...,2,1,0
2999,https://www.domain.com.au/5-wareham-street-spr...,5 Wareham Street Springvale VIC 3171,$515 Per Week,"[-37.9385223, 145.1558214]","[[3 Beds], [1 Bath], [2 Parking]]",House,Date Available: Available NowBond $2238,Positioned within walking distance to Springva...,3,1,2
3000,https://www.domain.com.au/3-34-hudson-street-c...,3/34 Hudson Street Coburg VIC 3058,$390 p/w,"[-37.7422322, 144.9628022]","[[2 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $1690,Centrally located in the heart of Coburg sits ...,2,1,1


In [360]:
def lowercase(text):
    text = text.lower()
    return text
house_data_combined['cost_text'] = house_data_combined['cost_text'].apply(lowercase)

In [361]:
def amount(text):
    num = re.findall(r'\d*[,.]?\d+[,.]?\d+',text)
    if len(num) == 0:
        return None
    elif len(num) == 1:       
        weekly_rent = float(num[0].replace(',',''))
    else:
        if('high' in text and 'low' in text):
            high = float(re.findall(r'\$\d+,?.\d+.?\d+ pw high',text)[0].split(' ')[0][1:].replace(',',''))
            low = float(re.findall(r'\$\d+,?.\d+.?\d+ pw low',text)[0].split(' ')[0][1:].replace(',',''))
            weekly_rent = (high + low)/2
        elif('per week' in text or 'pw' in text):
            num_text = re.findall(r'\$\d+,?.\d+.?\d+ pw', text)
            if(len(num_text) == 1):
                weekly_rent = float(num_text[0].split(' ')[0][1:].replace(',',''))
            else:
                num_text = re.findall(r'\$\d*[,.]?\d+[,.]?\d+pw', text)
                if(len(num_text) == 1):
                    weekly_rent = float(num_text[0][1:-2].replace(',',''))
                elif len(num_text) == 0:
                    weekly_rent = float(re.findall(r'\d*[,.]?\d+[,.]?\d+ per week', text)[0].split(' ')[0])
                else:
                    print("1")
                    print(text)
        else:
            return None

       
    if('annually' in text):
        rent = weekly_rent/52
    elif('month' in text):
        rent = weekly_rent/4
    else:
        rent = weekly_rent
    return rent
        
        


In [362]:
house_data_combined['amount'] = house_data_combined['cost_text'].apply(amount)

In [364]:
def postcode(text):
    postcode = re.findall(r'VIC \d+',text)
    if(len(postcode) == 0):
        return None
    else:
        return postcode[0].split(' ')[1]
   

In [365]:
house_data_combined['postcode'] = house_data_combined['name'].apply(postcode)

In [366]:
def bond(text):
    if(text is None):
        return None
    if('Bond' in text):
        amount = re.findall(r'Bond \$\d+',text)
        if (len(amount) == 1):
            amount = float(amount[0].split(' ')[1][1:])
            return amount
   
    else:
        return None


In [367]:
house_data_combined['bond'] = house_data_combined['external_information'].apply(bond)
house_data_combined

,link,name,cost_text,coordinates,rooms,house_type,external_information,desc,bedroom,bathroom,parking,amount,postcode,bond
0,https://www.domain.com.au/188b-kerferd-road-al...,188B Kerferd Road Albert Park VIC 3206,$670.00 per week,"[-37.8479747, 144.9520365]","[[1 Bed], [1 Bath], []]",Apartment / Unit / Flat,"Available from Friday, 9th September 2022",Nadia Barnes,1,1,0,670.0,3206,NaN
1,https://www.domain.com.au/3-34-leslie-street-r...,3/34 Leslie Street Richmond VIC 3121,$320 per week,"[-37.8132125, 145.0067341]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,"Available from Wednesday, 5th October 2022Bond...",Ella Vander Sluys,1,1,1,320.0,3121,1390.0
2,https://www.domain.com.au/20-12-dundas-street-...,20/12 Dundas Street Thornbury VIC 3071,$240.00,"[-37.7528368, 145.0029462]","[[1 Bed], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,Date Available: Available NowBond $1043,Kathryn Ambesi,1,1,1,240.0,3071,1043.0
3,https://www.domain.com.au/4-317-dandenong-road...,4/317 Dandenong Road Prahran VIC 3181,$430 per week,"[-37.859496, 145.0083698]","[[2 Beds], [1 Bath], [1 Parking]]",Apartment / Unit / Flat,"Available from Friday, 9th September 2022Bond ...",South Melbourne Leasing,2,1,1,430.0,3181,1868.0
4,https://www.domain.com.au/43-philipson-street-...,43 Philipson Street Albert Park VIC 3206,$680.00,"[-37.8479147, 144.9506225]","[[3 Beds], [1 Bath], []]",House,"Available from Thursday, 29th September 2022Bo...",Louisa Corke,3,1,0,680.0,3206,2955.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,https://www.domain.com.au/1-4-victor-street-su...,1/4 Victor Street Sunshine North VIC 3020,$370,"[-37.7777696, 144.8437958]","[[3 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $1607,This attractive home in a great location in Su...,3,1,1,370.0,3020,1607.0
2998,https://www.domain.com.au/24-david-street-brun...,24 David Street Brunswick VIC 3056,$575pw / $2499pcm,"[-37.7704915, 144.9627108]","[[2 Beds], [1 Bath], []]",House,Date Available: Available NowBond $2499,This spacious home offers a range of impressiv...,2,1,0,575.0,3056,2499.0
2999,https://www.domain.com.au/5-wareham-street-spr...,5 Wareham Street Springvale VIC 3171,$515 per week,"[-37.9385223, 145.1558214]","[[3 Beds], [1 Bath], [2 Parking]]",House,Date Available: Available NowBond $2238,Positioned within walking distance to Springva...,3,1,2,515.0,3171,2238.0
3000,https://www.domain.com.au/3-34-hudson-street-c...,3/34 Hudson Street Coburg VIC 3058,$390 p/w,"[-37.7422322, 144.9628022]","[[2 Beds], [1 Bath], [1 Parking]]",House,Date Available: Available NowBond $1690,Centrally located in the heart of Coburg sits ...,2,1,1,390.0,3058,1690.0


In [369]:
house_data_combined.to_csv("../data/curated/cleaned.csv")

In [376]:
import json
import pandas as pd
  
# Opening JSON file
f2 = pd.read_json('../data/raw/response.json')
f2["dateUpdated"].value_counts()
  


2020-03-13T07:53:17.483Z    1
2019-07-19T06:48:38.463Z    1
2020-01-25T06:19:35.56Z     1
2019-08-26T03:21:08.72Z     1
2019-09-16T02:48:09.79Z     1
                           ..
2021-06-05T11:32:23.26Z     1
2021-12-29T09:10:31.107Z    1
2021-05-17T13:21:23.547Z    1
2022-06-24T13:20:34.05Z     1
2019-10-26T03:51:24.14Z     1
Name: dateUpdated, Length: 100, dtype: int64